In [1]:
from src.rl.NatureDQN import NatureDQN
from src.rl.AtariAgent import AtariAgent
from src.rl.util import run_episode
from tqdm import tqdm
import gym

config = {'conv_layers': 3,
          'conv_units': [32, 64, 64],
          'filter_sizes': [8, 4, 3],
          'strides': [4, 2, 1],
          'state_frames': 4,
          'fc_layers': 1,
          'fc_units': [512],
          'in_width': 84,
          'in_height': 84,
          'discount': 0.99,
          'device': '/gpu:0',
          'lr': 0.00025,
          'opt_decay': 0.95,
          'momentum': 0.0,
          'opt_eps': 0.01,
          'clip_delta': 1.0,
          'tensorboard': False,
          'tensorboard_freq': 50,
          'ckpt': 0,
          'random_seed': 42,
          'hist_size': 1e6,
          'batch_size': 32,
          'eps': 1.0,
          
          }

env = gym.make('Breakout-v4')
config['num_actions'] = env.action_space.n
net = NatureDQN(config)
#net.load('src/rl/trained/breakout')
agent = AtariAgent(env, net, config)


/home/ralars/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
s = env.reset()
s.shape

(210, 160, 3)

In [3]:
run_episode(env, agent, render=True, render_delay=0.03)

1.0

In [4]:
def run_experiment(env, agent, epsilon_decay, n_episodes) -> list:
    rewards = []
    for i in tqdm(range(n_episodes)):
        agent.reset()
        sum_r = run_episode(env, agent, learn=True)
        rewards.append(sum_r)
        agent.eps *= epsilon_decay
        if i % 1e4 == 0:
            agent.sync_target()
    agent.epsilon = 0
    sum_r = run_episode(env, agent)
    print('Trained for ', n_episodes, ' episodes. Last episode achieved a reward of ', sum_r)     
    return rewards


run_experiment(env, agent, 9e-7, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:05<00:50,  5.59s/it]

 20%|██        | 2/10 [00:10<00:41,  5.23s/it]

 30%|███       | 3/10 [00:18<00:42,  6.03s/it]

 40%|████      | 4/10 [00:28<00:42,  7.07s/it]

...

 80%|████████  | 8/10 [01:08<00:17,  8.51s/it]

 90%|█████████ | 9/10 [01:14<00:08,  8.28s/it]

100%|██████████| 10/10 [01:22<00:00,  8.21s/it]

Trained for  10  episodes. Last episode achieved a reward of  2.0


[0.0, 0.0, 3.0, 3.0, 1.0, 1.0, 6.0, 5.0, 1.0, 2.0]